## Automating Detection of Non-Functioning Readers

- 2 hour window
- automate to choose last available observation 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import io


CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [2]:
string = ''' SELECT segments.analysis_id
               FROM bluetooth.segments
            EXCEPT
             SELECT DISTINCT f.analysis_id
               FROM ( SELECT observations.analysis_id,
                        observations.measured_timestamp
                       FROM bluetooth.observations
                      WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                               FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                               FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                               FROM bluetooth.segments))
                      ORDER BY observations.measured_timestamp DESC) f'''
df = pandasql.read_sql(pg.SQL(string), con)


In [39]:
broken_readers = []
for i in range(len(df['analysis_id'])):
    string = '''SELECT * FROM bluetooth.observations
                WHERE analysis_id = %d 
                ORDER BY measured_timestamp desc 
                LIMIT 1''' % list(df['analysis_id'])[i]
    row = pandasql.read_sql(pg.SQL(string), con)
    broken_readers.append(list(row.loc[0]))

broken_readers

[[828593778,
  5628170139321244370,
  1455710,
  1489,
  1489,
  1,
  'QU_BV',
  2,
  'EA_BV',
  Timestamp('2018-02-15 08:26:45'),
  4,
  3408904,
  1],
 [828629318,
  -120884493921328363,
  1455724,
  332,
  332,
  1,
  'EA_BV',
  2,
  'QU_BV',
  Timestamp('2018-02-15 08:13:30'),
  0,
  2360324,
  1],
 [678984505,
  106590517138277,
  1432978,
  5757,
  5757,
  1,
  'BR3',
  2,
  'BR4',
  Timestamp('2017-07-21 08:20:11'),
  0,
  0,
  2],
 [787149425,
  3586964457315845861,
  1454853,
  535,
  535,
  1,
  'QU_RO',
  2,
  'DU_RO',
  Timestamp('2017-12-17 17:57:21'),
  0,
  0,
  2],
 [828630601,
  6995651790643157431,
  1453680,
  288,
  288,
  1,
  'QU_BV',
  2,
  'QU_PA',
  Timestamp('2018-02-15 08:18:06'),
  0,
  2098184,
  1],
 [787142109,
  -13313431454755274,
  1454832,
  3001,
  3001,
  1,
  'DU_RO',
  2,
  'QU_RO',
  Timestamp('2017-12-17 18:44:11'),
  4,
  0,
  2],
 [828414222,
  5628170139321244370,
  1454311,
  1695,
  1695,
  1,
  'QU_BV',
  2,
  'KN_PA',
  Timestamp('2018-02

In [46]:
pd.DataFrame(broken_readers, columns = list(row.columns.values))

,id,user_id,analysis_id,measured_time,measured_time_no_filter,startpoint_number,startpoint_name,endpoint_number,endpoint_name,measured_timestamp,outlier_level,cod,device_class
0,828593778,5628170139321244370,1455710,1489,1489,1,QU_BV,2,EA_BV,2018-02-15 08:26:45,4,3408904,1
1,828629318,-120884493921328363,1455724,332,332,1,EA_BV,2,QU_BV,2018-02-15 08:13:30,0,2360324,1
2,678984505,106590517138277,1432978,5757,5757,1,BR3,2,BR4,2017-07-21 08:20:11,0,0,2
3,787149425,3586964457315845861,1454853,535,535,1,QU_RO,2,DU_RO,2017-12-17 17:57:21,0,0,2
4,828630601,6995651790643157431,1453680,288,288,1,QU_BV,2,QU_PA,2018-02-15 08:18:06,0,2098184,1
5,787142109,-13313431454755274,1454832,3001,3001,1,DU_RO,2,QU_RO,2017-12-17 18:44:11,4,0,2
6,828414222,5628170139321244370,1454311,1695,1695,1,QU_BV,2,KN_PA,2018-02-15 08:30:11,4,3408904,1
7,828628267,5439920273998709046,1453667,1030,1030,1,QU_PA,2,QU_BV,2018-02-15 08:01:21,4,7936,1
8,787049876,2447725881276873547,1412684,943,943,1,AD2,2,AD1,2017-12-17 18:00:14,0,0,2
9,787011361,-4122691999944392950,1412641,3197,3197,1,AD1,2,AD2,2017-12-17 18:47:53,4,0,2
